In [2]:
import pandas as pd
import numpy as np
import requests
import io
import zipfile
import time

pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

## Binance - Spot market data

Binance Spot prices and volumes extracted for the same time granularity (5 min) and for the relevant token pairs on AAVE

In [15]:
symbols = ['BTCUSDT','ETHUSDT','BUSDUSDT','USDTDAI','AAVEUSDT','LINKUSDT','CRVUSDT',]
url_base = "https://data.binance.vision/data/spot/monthly/klines/"

In [ ]:
dfs = pd.DataFrame()

for symbol in symbols:
    print(f'Doing {symbol}', end = (' ' * 10) + '\r')
    for year in range(2021, 2024):
        for month in range(1, 13):
            if year == 2023 and month > 2:
                continue
            file_url = f'{url_base}{symbol}/5m/{symbol}-5m-{year}-{month:02}.zip'
            response = requests.get(file_url)
            time.sleep(3)
            try:
                with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                    with z.open(f'{symbol}-5m-{year}-{month:02}.csv') as f:
                        df = pd.read_csv(f, header=None, names=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
                        df['symbol'] = symbol
                        dfs = pd.concat([dfs, df])
            except:
                continue

In [8]:
dfs['timestamp'] = pd.to_datetime(dfs.timestamp / 1000, unit='s')
dfs.set_index('timestamp', inplace = True)

In [9]:
dfs.head(2)

,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,symbol
timestamp,,,,,,,,,,,,
2021-01-01 00:00:00,28923.63,29017.50,28913.12,28975.65,182.89,1609459499999,5300786.60,5614,80.03,2319246.60,0,BTCUSDT
2021-01-01 00:05:00,28975.65,28979.53,28846.28,28858.94,214.57,1609459799999,6201531.66,4928,113.76,3287213.12,0,BTCUSDT


In [6]:
dfs.symbol.unique()

array(['BTCUSDT', 'ETHUSDT', 'USDCUSDT', 'USDTDAI', 'AAVEUSDT',
       'LINKUSDT', 'CRVUSDT'], dtype=object)

In [18]:
dfcum  = pd.DataFrame()
dfcum.shape

(0, 0)

In [22]:
dfcum  = pd.DataFrame()

for symbol, group in dfs.groupby('symbol'):
    # Calculate cummulative net variables
    group = group.loc[:,['close','quote_asset_volume','number_of_trades']]
    group.rename(columns={'close':f'{symbol}Price','quote_asset_volume':f'{symbol}volumeUsd',
                          'number_of_trades':f'{symbol}trades'}, inplace=True)
    if dfcum.shape[0] == 0:
        dfcum = group
    else:
        dfcum = dfcum.join(group)
print(dfcum.shape)
dfcum.head(2)

(227035, 21)


,AAVEUSDTPrice,AAVEUSDTvolumeUsd,AAVEUSDTtrades,BTCUSDTPrice,BTCUSDTvolumeUsd,BTCUSDTtrades,CRVUSDTPrice,CRVUSDTvolumeUsd,CRVUSDTtrades,ETHUSDTPrice,ETHUSDTvolumeUsd,ETHUSDTtrades,LINKUSDTPrice,LINKUSDTvolumeUsd,LINKUSDTtrades,USDCUSDTPrice,USDCUSDTvolumeUsd,USDCUSDTtrades,USDTDAIPrice,USDTDAIvolumeUsd,USDTDAItrades
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,88.25,77205.92,170,28975.65,5300786.60,5614,0.63,44753.02,116,736.89,2098521.63,1813,11.20,482219.04,931,1.00,284137.50,102.00,1.00,18906.15,9
2021-01-01 00:05:00,87.42,56698.56,213,28858.94,6201531.66,4928,0.62,156109.53,170,732.80,2086053.92,2463,11.11,520690.38,1164,1.00,134595.73,141.00,1.00,1311.92,6


In [24]:
dfcum.to_csv('binance-5min.csv')

_______________________________

## Coingecko Data

Aditional information aggregated by CoinGecko for relevant tokens:

In [ ]:
coingecko = pd.read_csv('coingecko-raw.csv', usecols=range(40))

In [83]:
coingecko = coingecko[coingecko.symbol.isin(['btc', 'eth', 'usdt', 'usdc', 'dai', 'aave', 'link', 'crv'])].reset_index(drop=True)
coingecko['date'] = pd.to_datetime(coingecko['date'].str[:15], format='%a %b %d %Y')

coingecko = coingecko.loc[:,['date','symbol','sentimentVotesUpPercentage','coingeckoScore','communityScore','marketData.circulatingSupply','communityData.redditAvgComments48h']]
coingecko.head()

,date,symbol,sentimentVotesUpPercentage,coingeckoScore,communityScore,marketData.circulatingSupply,communityData.redditAvgComments48h
0,2021-05-30,btc,58.9,80.48,71.28,18722012.00,1682.25
1,2021-05-30,eth,64.11,77.58,62.90,116086323.00,234.50
2,2021-05-30,usdt,69.54,41.77,10.72,61920512421.57,NaN
3,2021-05-30,usdc,82.35,49.59,NaN,22413102766.07,NaN
4,2021-05-30,link,73.88,64.12,48.09,429509553.92,9.17


In [85]:
print(coingecko.date.min())
print(coingecko.date.max())

2021-05-30 00:00:00
2023-03-26 00:00:00


In [7]:
coingecko = coingecko.rename(columns={'date':'timestamp'})
coingecko.set_index('timestamp', inplace = True)
coingecko.head(2)

,symbol,sentimentVotesUpPercentage,coingeckoScore,communityScore,marketData.circulatingSupply,communityData.redditAvgComments48h
timestamp,,,,,,
2021-05-30,btc,58.9,80.48,71.28,18722012.00,1682.25
2021-05-30,eth,64.11,77.58,62.90,116086323.00,234.50
2021-05-30,usdt,69.54,41.77,10.72,61920512421.57,NaN
2021-05-30,usdc,82.35,49.59,NaN,22413102766.07,NaN
2021-05-30,link,73.88,64.12,48.09,429509553.92,9.17


In [33]:
dfcum  = pd.DataFrame()

for symbol, group in coingecko.groupby('symbol'):
    # Calculate cummulative net variables
    symbol = symbol.upper()
    group = group.iloc[:,1:]
    group.rename(columns={'sentimentVotesUpPercentage':f'{symbol}sentimentUp','coingeckoScore':f'{symbol}coingeckoScore',
                          'communityScore':f'{symbol}communityScore','marketData.circulatingSupply':f'{symbol}circulating',
                          'communityData.redditAvgComments48h':f'{symbol}redditComments'}, inplace=True)
    if dfcum.shape[0] == 0:
        dfcum = group
    else:
        dfcum = dfcum.join(group)
print(dfcum.shape)
dfcum.head(2)

(857, 40)


,AAVEsentimentUp,AAVEcoingeckoScore,AAVEcommunityScore,AAVEcirculating,AAVEredditComments,BTCsentimentUp,BTCcoingeckoScore,BTCcommunityScore,BTCcirculating,BTCredditComments,CRVsentimentUp,CRVcoingeckoScore,CRVcommunityScore,CRVcirculating,CRVredditComments,DAIsentimentUp,DAIcoingeckoScore,DAIcommunityScore,DAIcirculating,DAIredditComments,ETHsentimentUp,ETHcoingeckoScore,ETHcommunityScore,ETHcirculating,ETHredditComments,LINKsentimentUp,LINKcoingeckoScore,LINKcommunityScore,LINKcirculating,LINKredditComments,USDCsentimentUp,USDCcoingeckoScore,USDCcommunityScore,USDCcirculating,USDCredditComments,USDTsentimentUp,USDTcoingeckoScore,USDTcommunityScore,USDTcirculating,USDTredditComments
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-30,77.78,51.63,39.86,12783535.25,1.83,58.9,80.48,71.28,18722012.00,1682.25,92.86,48.77,10.82,366407235.61,NaN,63.89,40.36,43.64,4437337717.39,2.92,64.11,77.58,62.90,116086323.00,234.50,73.88,64.12,48.09,429509553.92,9.17,82.35,49.59,NaN,22413102766.07,NaN,69.54,41.77,10.72,61920512421.57,NaN
2021-06-01,87.29,51.76,39.63,12786393.16,1.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,39.76,42.91,4407750535.75,5.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
dfcum.to_csv('coingecko-proc.csv')

____________________________

## Tweets Data

From a Bigquery table, where data from Twitter referring to +1000 Crypto accounts from 2017 onwards are stored, mentions are gathered referring to the tickets that interest us in this analysis are extracted, for the same temporal dimensionality that we have been working on (5 minutes ) from where we get:

- The number of tweets with that mention
- The sum of the responses of said tweets
- The sum of the retweets of said tweets
- The sum of the likes of said tweets

In [ ]:
from google.cloud import bigquery

client = bigquery.Client.from_service_account_json('crypto-analysis.json')


In [ ]:
query = """
WITH recount AS(
  SELECT
    TIMESTAMP_TRUNC(timestamp, HOUR) + INTERVAL (EXTRACT(MINUTE FROM timestamp) - MOD(EXTRACT(MINUTE FROM timestamp), 5)) MINUTE AS timestamp,
    REGEXP_EXTRACT(text, '\\$[A-Za-z]{3,5}') AS ticket,
    COUNT(*) AS count,
    SUM(repliesCount) AS repliesCount,
    SUM(retweetsCount) AS retweetsCount,
    SUM(likesCount) AS likesCount,
  FROM `crypto-mauna.twitter.tweets` 
  WHERE
    REGEXP_EXTRACT(text, '\\$[A-Za-z]{3,5}') IS NOT NULL
    AND date(timestamp) > date("2021-01-01")
  GROUP BY
    1, 2
  ORDER BY
    1, 3 DESC
)

select
  *
from recount
where ticket in ('$BTC', '$ETH', '$USDT', '$USDC', '$DAI', '$AAVE', '$LINK', '$CRV')
"""

In [ ]:
query_job = client.query(query)
results = query_job.result()

In [ ]:
results.to_csv('tweets-proc.csv', index = False)